# ANALYZE HEART ACTIVITY: BOUTS AND FREQUENCY

# IMPORT all the modules you will need

In [2]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from scipy import fft
from scipy import signal
import pandas as pd
import os

from scipy.signal import argrelextrema

import operator

import seaborn as sns; sns.set()

import math
import matplotlib.patches as patches

from pandas import DataFrame
from scipy.interpolate import interp1d

from scipy.signal import decimate

from operator import itemgetter
from itertools import groupby


from itertools import chain
from scipy import stats

from scipy.stats import ks_2samp
from scipy.stats.stats import pearsonr 



# Define path and frequence 

In [3]:

path ='/Users/Natalia/Documents/DATA/Analysis w- tinDs 1do/'


fq= 29.98   #frames per second

f=60.03 # frames per second interpolated signal

factor = 2.0023   # to get the interpolated position of loomings

reso=300  # PLots resolution to save

# Define position of the fake loomings during baseline and stimuli

In [4]:
#times = [300, 283, 269, 250, 231, 213, 198, 186, 169, 155, 140, 126, 112, 101,  89,  71, 57,  43,  26,  13]
#sum_sec= [0,  17,  31,  50,  69,  87,  102, 114, 131, 145, 160, 174, 188, 199, 211, 229, 243, 257, 274, 287]
#sec  =[0,  17,   14,  19,  19,  18,  15,  12,  17,   14,  15,  14, 14,  11,  12,  18,  14,  14,  17,  13]
    
fake_sec= [2,  17,  31,  50,  69,  87,  102, 114, 131, 145, 160, 174, 188, 199, 211, 229, 243, 257, 274, 287]
fake_ini =[]
for fi in fake_sec:
    fi= fi*f
    fake_ini.append(int(round(fi)))

fake_end =[]
for fj in fake_sec:
    fj= (fj+0.5)*f
    fake_end.append(int(round(fj)))
fake_position = pd.DataFrame(data={'f_ending': fake_end, 'f_starting': fake_ini})


  
loom_sec= [300,  317,  331,  350,  369,  387,  402, 414, 431, 445, 460, 474, 488, 499, 511, 529, 543, 557, 574, 587]
ini =[]
for i in loom_sec:
    i= i*f
    ini.append(int(round(i)))

end =[]
for j in loom_sec:
    j= (j+0.5)*f
    end.append(int(round(j)))
position = pd.DataFrame(data={'ending': end, 'starting': ini})
#position


ini_fake =[]
for i in loom_sec:
    i= (i+7)*f
    ini_fake.append(int(round(i)))

end_fake =[]
for j in loom_sec:
    j= (j+7.5)*f
    end_fake.append(int(round(j)))
position_fake = pd.DataFrame(data={'ending': end_fake, 'starting': ini_fake})
#position_fake

  

# Read velocity, movement and cut them in 601 s

In [5]:
# Get starting poing of the experiment from the LED
Light = [] 
for item in os.listdir(path):
    if item.startswith('Stimulus'):
        light = pd.read_csv(item,' ', header = None)
        frames_light= light.loc[light[1] > 5000, 0]
        frames_light=frames_light.tolist()
        #frames_light= light.pop(1)
        Light.append(frames_light)




In [6]:
# Read Velocity
x=0
Velocity = [] 
for item in os.listdir(path):
    if item.startswith('BallActivity'):
        ball_act = pd.read_csv(item,' ', header = None)
        ball_extra = ball_act.pop(3)        
        ball_act= pd.DataFrame({'X':ball_act[0],'Y':ball_act[1],'Z':ball_act[2]}) 
        velocity = np.sqrt(ball_act['X']**2 + ball_act['Y']**2 + ball_act['Z']**2)  # We calculate speed with the vector in the three directions
        vel_aver= velocity.rolling(5, center=True).mean()                        # I'm doing a rolling mean!!!!!
        a= Light[x]                             #Cut the files acording to the first frame with LED light
        b= vel_aver[a[0]:a[0]+36078].tolist()
        Velocity.append(b)
        x=x+1
        


In [7]:
#Read leg movements
x=0
Leg_mov= [] 
for item in os.listdir(path):
    if item.startswith('FlyActivity'):
        fly_act = pd.read_csv(item,' ', header = None) 
        leg_act= fly_act.drop([0,2,3], axis=1)      
        leg_aver= leg_act.rolling(5, center=True).mean()    # I'm doing a rolling mean!!!!!
        leg_a=leg_aver.values.tolist() 
        flattened = []
        for sublist in leg_a:
            for val in sublist:
                flattened.append(val/1)
        a= Light[x]
        b= flattened[a[0]:a[0]+36078]                         #Cut the files acording to the first frame with LED light 
        Leg_mov.append(b)
        x=x+1


# READ THE X AXIS

two cells

In [8]:
#Read X files, filling the nan 
x_files = []

for item in os.listdir(path):
    if item.startswith('X_Y_mov'):
        df_x = pd.read_csv(item,' ', header = None)
        df_xa = df_x.pop(0)    # remove one columm extra 
        df_xp = df_x.pop(2)
        df_xa = np.array(df_xa)
        oka = ~np.isnan(df_xa)
        xpa = oka.ravel().nonzero()[0]
        fpa = df_xa[~np.isnan(df_xa)]
        xa  = np.isnan(df_xa).ravel().nonzero()[0]
        df_xa[np.isnan(df_xa)] = np.interp(xa, xpa, fpa)   
        df_xa -= np.nanmean(df_xa); df_xa /= np.nanstd(df_xa)   # regularize datasets by subtracting mean and dividing by s.d.    
        x_files.append(df_xa)
        
        df_xp = np.array(df_xp)
        okp = ~np.isnan(df_xp)
        xpp = okp.ravel().nonzero()[0]
        fpp = df_xp[~np.isnan(df_xp)]
        xp  = np.isnan(df_xp).ravel().nonzero()[0]
        df_xp[np.isnan(df_xp)] = np.interp(xp, xpp, fpp)
        df_xp -= np.nanmean(df_xp); df_xp /= np.nanstd(df_xp)   # regularize datasets by subtracting mean and dividing by s.d. 
        x_files.append(df_xp)


# Interpolate X signal

In [10]:
def interpol(wave):
    frames = np.arange(0,len(wave))

    y_interp = interp1d(frames, wave, kind='cubic', bounds_error=False) 

    new_frames = np.arange(0, len(wave), len(wave)/36078)
    
    return y_interp(new_frames)



In [11]:
Heart=[]
for ani in x_files:
    heart= interpol(ani)
    Heart.append(heart)



# IDENTIFICATION of BACKWARD AND FOREWARD BEATINGS

In [12]:
def peaks_less(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
        
    
    peaksmax_A = np.array(argrelextrema(waveA, np.less_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.less_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]


    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A       # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     

 

    PeaksPList = [0] * P #=len(waveP)  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      
   

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
 
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    

    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)

    
    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':     
                list_backward.append(i)

              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
        
                                
    print 'num of backward beatings=', len(list_backward)                                                                 

    return list_backward, list_foreward




In [13]:
def peaks_greater(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
           
    peaksmax_A = np.array(argrelextrema(waveA, np.greater_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.greater_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A #=len(waveA)  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     



    PeaksPList = [0] * P  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      
   
    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    

    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)


    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':   
                list_backward.append(i)
                #pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:# and Listpeaks[i-4]!= 'post':
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
                                
    print 'num of backward beatings=', len(list_backward)                                                                 

    return list_backward, list_foreward


In [14]:
# Separate in different arrays each backward pulse and each foreward pulse

def direction(list_1, list_2):
    "Separate in different arrays each backward pulse and each foreward pulse"
    "these lists only contain indexes for the peaks"
    portions= np.split(list_1,np.searchsorted(list_1, list_2))
   
    return portions


In [15]:
# Get sequences only of the baseline or stimulation for heart data
Heart_total=[]
for ani in Heart:
    base= ani[0:]
    Heart_total.append(base)


Heart_base=[]
for ani in Heart:
    base= ani[0:18010]
    Heart_base.append(base)

    
Heart_stim=[]
for ani in Heart:
    stim= ani[18010:]
    Heart_stim.append(stim)



    

In [ ]:
# Run the function peaks for each animal and got peaks according to direction in anterior cell

#During whole experiment
backward_pulses_t=[]

for ani in range(0,len(Heart_total),2):  # Identify high and lower peaks and add them
    print ani//2
    back1, fore1, = peaks_greater(Heart_total[ani],Heart_total[ani+1],A=36078,P=36078)
    back2, fore2= peaks_less(Heart_total[ani],Heart_total[ani+1],A=36078,P=36078)   

    fore3=fore1+fore2
    back3=back1+back2
    w= sorted(fore3)
    y= sorted(back3)

    backward=direction(y,w)
    back_clean=[]
    for each in backward:
        if len(each)>2: 
            for e in each:
                back_clean.append(e)
    backward_pulses_t.append(back_clean) 

        

In [19]:
# Remove those peaks that are too close from the previous one


for ani in backward_pulses_t:
    bck_deleted=[]
    for i in range(0,1):
        if ani[i+1]< ani[i]+3:       
            bck_deleted.append(i)

    for i in range(1, len(ani)-1):
        if ani[i+1]< ani[i]+3:     
            bck_deleted.append(i)

    for i in range(len(ani)-1, len(ani)):
        if ani[i]< ani[i-1]+3:       
            bck_deleted.append(i)
    

    for index in sorted(bck_deleted, reverse=True):
        del ani[index]


In [ ]:
# Slipt the peaks in fragments according to the distance among them

t_periods_backward=[]
for ani in backward_pulses_t:
    back_frag=[]
    array=[]
    array1=[]

    for i in range(0, len(ani)-1):
        if ani[i]+18 >= ani[i+1]:
            array.append(ani[i])
    
        else:
            array.append(ani[i])
            back_frag.append(array)
            array=[]
    
    back_per=[]       
    for x in back_frag:             # Remove those fragments with less tham 3 peaks.
        if len(x)>3:
            back_per.append(x)
    t_periods_backward.append(back_per)       
print len(t_periods_backward)


In [24]:
def peaks_less1(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase but excluding anpo peaks from backward list"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
        
    
    peaksmax_A = np.array(argrelextrema(waveA, np.less_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.less_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     


    PeaksPList = [0] * P  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
 
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    

    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)

    
    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':
                pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
        
                                
    print 'num of backward beatings=', len(list_backward)                                                                 

    return list_backward, list_foreward



In [25]:
def peaks_greater1(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase but excluding anpo peaks from backward list"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
           
    peaksmax_A = np.array(argrelextrema(waveA, np.greater_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.greater_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     

   
    PeaksPList = [0] * P   # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    
  
    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)


    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':
                pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:# and Listpeaks[i-4]!= 'post':
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
                                
    print 'num of backward beatings=', len(list_backward)                                                                 
 
    return list_backward, list_foreward


In [26]:
# Manually selection of animals to be analysed excluding simultaneous peaks

# second list to analyze with peaks_1
del_beh=[3,6,9,10,12,17,18,19,20,21,22,26,27,28,29,30,31,32,33,34,36,37,38,39,43,46,47,48,49,50,52,59,60,61,64,66,69,70,71,72,74,75,76,77,78,79,81,82,84,87,88,89,90,91,94,96,97,98,101,102,103,104,105,106,107,108,109,112,113,114,115]

#animals to remove from the Heart_total list
del_beh_b=[3,6,9,10,12,17,18,19,20,21,22,26,27,29,30,31,32,33,36,37,38,39,43,46,47,48,49,50,52,59,60,61,64,66,69,70,71,72,74,75,76,77,78,79,81,82,84,87,88,89,90,91,94,96,97,98,101,102,103,104,105,106,107,108,109,112,113,114,115]

Heart_total_2=[]
Heart_base_2=[]
Heart_stim_2=[]
for ani in del_beh:
    Heart_base_2.append(Heart_base[ani*2])
    Heart_base_2.append(Heart_base[(ani*2)+1])
    Heart_stim_2.append(Heart_stim[ani*2])
    Heart_stim_2.append(Heart_stim[(ani*2)+1])
    Heart_total_2.append(Heart_total[ani*2])
    Heart_total_2.append(Heart_total[(ani*2)+1])
    

In [ ]:
# Run the function peaks for each  animal and got peaks according to direction

#During whole experiment
backward_pulses_t2=[]
#Listpeaks2=[]
#Listvalleys2=[]
for ani in range(0,len(Heart_total_2),2):  # Identify high and lower peaks and add them
    print ani//2
    back1, fore1= peaks_greater1(Heart_total_2[ani],Heart_total_2[ani+1],A=36078,P=36078)
    back2, fore2= peaks_less1(Heart_total_2[ani],Heart_total_2[ani+1],A=36078,P=36078)   
    
    fore3=fore1+fore2
    back3=back1+back2
    w= sorted(fore3)
    y= sorted(back3)
    
    
    backward=direction(y,w)
    back_clean=[]
    for each in backward:
        if len(each)>2: 
            for e in each:
                back_clean.append(e)
    backward_pulses_t2.append(back_clean) 

In [29]:
# Remove those peaks that are too close from the previous one


for ani in backward_pulses_t2:
    bck_deleted=[]
    for i in range(0,1):
        if ani[i+1]< ani[i]+3:       
            bck_deleted.append(i)

    for i in range(1, len(ani)-1):
        if ani[i+1]< ani[i]+3:    
            bck_deleted.append(i)

    for i in range(len(ani)-1, len(ani)):
        if ani[i]< ani[i-1]+3:       
            bck_deleted.append(i)
    
    #print bck_deleted
    for index in sorted(bck_deleted, reverse=True):
        del ani[index]

In [ ]:
# Split the peaks in fragments according to the distance among them

t_periods_backward_2=[]
for ani in backward_pulses_t2:
    back_frag=[]
    array=[]
    array1=[]

    for i in range(0, len(ani)-1):
        if ani[i]+18 >= ani[i+1]:
            array.append(ani[i])
    
        else:
            array.append(ani[i])
            back_frag.append(array)
            array=[]
            
        
    back_per=[]       
    for x in back_frag:             # Remove those fragments with less tham 3 peaks.
        if len(x)>3:
            back_per.append(x)
    t_periods_backward_2.append(back_per)       



In [ ]:
#Manually define files from  heart_2 list to discard 

del_beh_2=[0,12,13,14,15,16,17,19,20,22,23,24,30,32,37,38,41,43,48,49,51,53,54,56,58,61,62,67]
del_hearts_2=[]
for each in del_beh_2:
    del_hearts_2.append(each*2)
    del_hearts_2.append((each*2)+1)

print del_hearts_2

# Get arrays of 10 and -10 to represent forward and backward periods 

In [45]:
## (10 FORWARD  and -10 BACKWARD)
    
zeros_total=[]
for ani in t_periods_backward_new:
    ani0=[10]*36068
    for each in ani:
        for num in range(each[0],each[-1]):
            ani0[num]=-10
    zeros_total.append(ani0)


# GET PERIODS WHEN FLIES ARE RUNNING GROOMING OR FREEZING

In [46]:
#Return periods of immobilization during baseline and stimulation according to ball movement

def moving_ball(list):
    "return the indexes of the periods longer than 2.5 sec where the fly is NOT walking during baseline and stimulation"
    nowalkbase=[]
    nowalkstim=[]
    for i, j in enumerate(list):
        if i<18010 and j < 0.5:#1.0:
            nowalkbase.append(i)
        else:
            if 36078>i>= 18010 and j<0.5:#1.0:
                nowalkstim.append(i)
                
    quietbase=[]
    for k, g in groupby(enumerate(nowalkbase), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietbase.append(stop)

    quietstim=[]
    for k, g in groupby(enumerate(nowalkstim), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietstim.append(stop)
    
    nowalkingbase=[]
    nowalkingstim=[]
    for array in quietbase:
        if len(array)>150:   # 2.5 seconds
            nowalkingbase.append(array)
        else:
            pass
    for array in quietstim:
        if len(array)>150:
            nowalkingstim.append(array)
        else:
            pass
    print len(nowalkingbase),len(nowalkingstim)
    return nowalkingbase, nowalkingstim

In [47]:
# Return freezing periods during baseline and stimulation according to legs movement
def freezing_legs(list):
    "return the indexes of the periods longer than 2.5 sec where the fly is immobile during baseline and stimulation"
    freezbase=[]
    freezstim=[]
    for i, j in enumerate(list):
        if i<18010 and np.mean(list[i-5:i+5]) < 5:
            freezbase.append(i)
        else:
            if 36078>i>= 18010 and np.mean(list[i-5:i+5]) < 5:
                freezstim.append(i)
    
    quietbase=[]
    for k, g in groupby(enumerate(freezbase), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietbase.append(stop)

    quietstim=[]
    for k, g in groupby(enumerate(freezstim), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietstim.append(stop)
    
    freezingbase=[]
    freezingstim=[]
    for array in quietbase:
        if len(array)>150:    # 2.5 seconds
            freezingbase.append(array)
        else:
            pass
    for array in quietstim:
        if len(array)>150:
            freezingstim.append(array)
        else:
            pass
    print len(freezingbase),len(freezingstim)
    return freezingbase, freezingstim

In [ ]:
# Identify no walking periods in Velocity file  #GET LIST OF INDEXES 


NoWalkbase=[]     
NoWalkstim=[]

for ani in Velocity_new:
    nowalkingbase, nowalkingstim = moving_ball(ani)
    NoWalkbase.append(nowalkingbase)
    NoWalkstim.append(nowalkingstim)
    


In [ ]:
# Identify moving periods in leg file  # GET LIST OF INDEXES 

Freezbase=[]
Freezstim=[]   

for ani in Leg_mov_new:
    freezingbase, freezingstim = freezing_legs(ani)
    Freezbase.append(freezingbase)
    Freezstim.append(freezingstim)
     


# separate freezers from runners according to time freezing during stimulation

In [ ]:
# Time freezing

flat_Freezstim=[]
for ani in Freezstim[0:90]:
    b= list(chain.from_iterable(ani))
    flat_Freezstim.append(len(b))


In [ ]:
freezers=[]
runners=[]
mixers=[]
for i, e in enumerate(flat_Freezstim[27:]):
    if e> 12600:
        freezers.append(i+27)
    elif e<5400:
        runners.append(i+27)
    else: 
        mixers.append(i+27)
